In [1]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/rana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any'

In [9]:
from ast import literal_eval
import pandas as pd
import numpy as np

In [10]:
train = pd.read_csv("train.csv")
test = pd.read_csv('test.csv')
X_train, y_train = train['text'].values, train['author'].values
#X_val, y_val = validation['title'].values, validation['tags'].values
X_test = test['text'].values

In [14]:
print(type(y_train))

<class 'numpy.ndarray'>


In [6]:
import re

In [7]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    #print(text)
    text = text.lower() # lowercase text
    #print(text)
    text = re.sub(REPLACE_BY_SPACE_RE, " ", text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    # print(text)
    text = re.sub(BAD_SYMBOLS_RE, "", text) # delete symbols which are in BAD_SYMBOLS_RE from text
    # print(text)
    text = " " + text + " "
    for sw in STOPWORDS:
        text = text.replace(" "+sw+" ", " ") # delete stopwors from text
    #print(text)
    text = re.sub('[ ][ ]+', " ", text)
    length=len(text)-1
    #print(length)
    
    if text[0] == ' ':
        text = text[1:]
    if length == 0:
        return text
    if text[length-1] == ' ':
        text = text[:-1]
        
    # text = text[1:-1]
    # print(text)
    return text

In [8]:
X_train = [text_prepare(x) for x in X_train]
#X_val = [text_prepare(x) for x in X_val]
X_test = [text_prepare(x) for x in X_test]

In [9]:
print(X_train[-1])

laid gnarled claw shoulder seemed shaking altogether mirth


In [11]:
# Dictionary of all tags from train corpus with their counts.
from collections import Counter
# Dictionary of all tags from train corpus with their counts.
tags_counts = Counter() #{}
# Dictionary of all words from train corpus with their counts.
words_counts = Counter() #{}

######################################
######### YOUR CODE HERE #############
######################################
# print(X_train[:3], y_train[:3])
for sentence in X_train:
    for word in sentence.split():
        # print(word)
        words_counts[word] += 1


for l in y_train:
        tags_counts[l] += 1

In [12]:
most_common_tags = sorted(tags_counts.items(), key=lambda x: x[1], reverse=True)[:]
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:]

In [13]:
print(sorted(tags_counts.items(), key=lambda x: x[1], reverse=True)[:])

[('EAP', 7900), ('MWS', 6044), ('HPL', 5635)]


In [14]:
print(most_common_tags)

[('EAP', 7900), ('MWS', 6044), ('HPL', 5635)]


In [15]:
print(len(most_common_words),type(most_common_words))

25317 <class 'list'>


In [16]:
print(most_common_words)

[('one', 1614), ('upon', 1411), ('could', 1316), ('would', 1241), ('man', 730), ('time', 729), ('yet', 715), ('said', 704), ('even', 701), ('might', 629), ('old', 616), ('like', 613), ('first', 602), ('us', 595), ('must', 594), ('never', 570), ('made', 565), ('life', 562), ('night', 559), ('found', 559), ('seemed', 544), ('eyes', 540), ('every', 535), ('little', 531), ('still', 519), ('day', 516), ('great', 512), ('long', 511), ('saw', 502), ('many', 491), ('well', 475), ('came', 461), ('much', 459), ('may', 442), ('thought', 441), ('two', 431), ('see', 411), ('say', 409), ('ever', 408), ('thus', 405), ('whose', 401), ('death', 380), ('without', 377), ('mind', 377), ('far', 376), ('things', 371), ('heart', 370), ('shall', 368), ('heard', 366), ('house', 365), ('however', 358), ('men', 355), ('left', 354), ('thing', 354), ('years', 347), ('felt', 343), ('place', 339), ('last', 338), ('know', 337), ('love', 331), ('light', 320), ('though', 318), ('indeed', 316), ('come', 314), ('world', 

In [17]:
print(len(most_common_words))

25317


In [18]:
DICT_SIZE = 5000
WORDS_TO_INDEX = {p[0]:i for i,p in enumerate(most_common_words[:DICT_SIZE])} ####### YOUR CODE HERE #######
INDEX_TO_WORDS = {WORDS_TO_INDEX[k]:k for k in WORDS_TO_INDEX}####### YOUR CODE HERE #######
ALL_WORDS = WORDS_TO_INDEX.keys()

def my_bag_of_words(text, words_to_index, dict_size):
    """
        text: a string
        dict_size: size of the dictionary
        
        return a vector which is a bag-of-words representation of 'text'
    """
    result_vector = np.zeros(dict_size)
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    for word in text.split():
        if word in words_to_index:
            result_vector[words_to_index[word]] += 1
    return result_vector

In [19]:
from scipy import sparse as sp_sparse
    

In [21]:
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
#X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
X_test_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_test])
print('X_train shape ', X_train_mybag.shape)
#print('X_val shape ', X_val_mybag.shape)
print('X_test shape ', X_test_mybag.shape)

print(most_common_words[:10])
for w in X_train[10].split():
    print(w)
    print(w in WORDS_TO_INDEX)

X_train shape  (19579, 5000)
X_test shape  (8392, 5000)
[('one', 1614), ('upon', 1411), ('could', 1316), ('would', 1241), ('man', 730), ('time', 729), ('yet', 715), ('said', 704), ('even', 701), ('might', 629)]
shall
True
find
True
feel
True
injuries
False
shall
True
learn
True
dread
True
revenge
True
days
True
arrived
True


In [22]:
x_train=np.array(X_train_mybag.toarray())

In [23]:
x_test=np.array(X_test_mybag.toarray())

In [25]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(19579, 5000)
(8392, 5000)
(19579,)


In [41]:
x_train[0,:25]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0.])

In [15]:
y_train_hot=np.array(y_train)
for i in range(0 ,len(y_train)):
    if y_train[i]=="EAP":
        y_train_hot[i]=0
    elif y_train[i]=="HPL":
        y_train_hot[i]=1
    elif y_train[i]=="MWS":
        y_train_hot[i]=2

In [16]:
print(y_train[0:10])
print(y_train_hot[0:10])

['EAP' 'HPL' 'EAP' 'MWS' 'HPL' 'MWS' 'EAP' 'EAP' 'EAP' 'MWS']
[0 1 0 2 1 2 0 0 0 2]


In [18]:
print(type(y_train_hot))

<class 'numpy.ndarray'>


In [21]:
np.save("y_train.npy",y_train)

In [28]:
print(y_train[-5:])

['EAP' 'EAP' 'EAP' 'EAP' 'HPL']


In [29]:
print(y_train_hot[-5:])

[0 0 0 0 1]


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
def tfidf_features(X_train, X_test):
    """
        X_train, X_val, X_test — samples        
        return TF-IDF vectorized representation of each sample and vocabulary
    """
    # Create TF-IDF vectorizer with a proper parameters choice
    # Fit the vectorizer on the train set
    # Transform the train, test, and val sets and return the result
     
    tfidf_vectorizer = TfidfVectorizer(token_pattern='(\S+)', min_df=5, max_df=0.9, ngram_range=(1,2)) ####### YOUR CODE HERE #######
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    tfidf_vectorizer.fit(X_train)
    X_train = tfidf_vectorizer.transform(X_train)
    X_test = tfidf_vectorizer.transform(X_test)
    print(tfidf_vectorizer.vocabulary_)
    return X_train , X_test, tfidf_vectorizer.vocabulary_

In [32]:
X_train_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_test)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}

{'restore': 6869, 'flowers': 3204, 'connection': 1568, 'mud': 5372, 'terminated': 8243, 'wax': 9095, 'combat': 1434, 'offended': 5681, 'apply': 390, 'sail': 7097, 'nevertheless': 5515, 'would bring': 9338, 'could well': 1750, 'conquered': 1571, 'parks': 5925, 'oaks': 5612, 'cosmos': 1690, 'arts': 468, 'idols': 4061, 'merely': 5137, 'fenner': 3087, 'glass': 3527, 'moral': 5305, 'extra': 2926, 'solid': 7636, 'retiring': 6887, 'yet time': 9443, 'perspective': 6064, 'cool': 1664, 'grounds': 3668, 'hundred': 4016, 'bent': 790, 'man ever': 4970, 'horrid': 3961, 'proud': 6459, 'code': 1405, 'led': 4655, 'remains': 6750, 'horrible': 3959, 'superhuman': 8038, 'reanimated': 6632, 'occasional': 5653, 'bitterness': 837, 'evilly': 2812, 'fog': 3220, 'wal': 9034, 'assigned': 503, 'blue eyes': 887, 'abnormal': 15, 'enveloped': 2712, 'madhouse': 4926, 'vivacity': 8990, 'liable': 4708, 'justine': 4472, 'cape': 1122, 'man would': 4977, 'englands': 2670, 'reserved': 6834, 'sounding': 7684, 'fruiterer': 3

In [33]:
x_train_tfidf=np.array(X_train_tfidf.toarray())
x_test_tfidf=np.array(X_test_tfidf.toarray())

In [35]:
print(len(tfidf_vocab))
print(tfidf_vocab)

9475
{'restore': 6869, 'flowers': 3204, 'connection': 1568, 'mud': 5372, 'terminated': 8243, 'wax': 9095, 'combat': 1434, 'offended': 5681, 'apply': 390, 'sail': 7097, 'nevertheless': 5515, 'would bring': 9338, 'could well': 1750, 'conquered': 1571, 'parks': 5925, 'oaks': 5612, 'cosmos': 1690, 'arts': 468, 'idols': 4061, 'merely': 5137, 'fenner': 3087, 'glass': 3527, 'moral': 5305, 'extra': 2926, 'solid': 7636, 'retiring': 6887, 'yet time': 9443, 'perspective': 6064, 'cool': 1664, 'grounds': 3668, 'hundred': 4016, 'bent': 790, 'man ever': 4970, 'horrid': 3961, 'proud': 6459, 'code': 1405, 'led': 4655, 'remains': 6750, 'horrible': 3959, 'superhuman': 8038, 'reanimated': 6632, 'occasional': 5653, 'bitterness': 837, 'evilly': 2812, 'fog': 3220, 'wal': 9034, 'assigned': 503, 'blue eyes': 887, 'abnormal': 15, 'enveloped': 2712, 'madhouse': 4926, 'vivacity': 8990, 'liable': 4708, 'justine': 4472, 'cape': 1122, 'man would': 4977, 'englands': 2670, 'reserved': 6834, 'sounding': 7684, 'fruitere

In [36]:
x_train_tfidf.shape

(19579, 9475)

In [72]:
np.save("x_train_tfidf.npy",x_train_tfidf)

In [3]:
x_train_tfidf=np.load("x_train_tfidf.npy")

In [4]:
print(x_train_tfidf.shape)

(19579, 9475)


##machine learning part

In [2]:
from sklearn.svm import SVC
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [3]:
x_train_frequency = np.load("x_train_binary.npy")
#x_train_tfidf = np.load("x_train_tfidf.npy")
y_train_labels = np.load("y_train_hot.npy")
x_test_frequency = np.load("x_test_binary.npy")
#x_test_tfidf = np.load("x_test_tfidf.npy")

In [4]:
X_train,Y_train=shuffle(x_train_frequency,y_train_labels, random_state=2)

In [5]:
x_train,x_test,y_train,y_test=train_test_split(X_train,Y_train,test_size=0.2,random_state=4)

MemoryError: 

In [1]:
clf=SVC(gamma='0.01')

NameError: name 'SVC' is not defined

In [7]:
y_train=y_train.astype('int')
y_test = y_test.astype("int")

In [10]:

clf.fit(x_train,y_train)
print("accuracy of model on test dataset",clf.score(x_test,y_test))

TypeError: a float is required

In [34]:
y=np.load("y.npy")

In [36]:
print(y.shape)

(3190,)


In [1]:
f=open("hello.txt",'x')

In [ ]:
f.w